In [101]:
# Import Library
import pandas as pd 
import numpy as np

current = pd.read_csv('current.csv', index_col="sasdate")
month = pd.read_excel('PredictorData2022.xlsx', index_col = 'yyyymm')
markus = pd.read_csv('Macro.csv', index_col = 'Date')

firm = pd.read_pickle("7_chars_data.pkl")

/Users/scarlett/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


##  "current_transformed.csv" Data Validation

In [41]:
class MacroCollect():
    def __init__(self,):
        self.variable = ['dp','ep','b/m','ntis','tbl','tms','dfy','svar',]
        self.tcodes = [2,2,5,2,2,1,2,5]
    
    def transxf(self, x, tcode) :
        n=x.size
        small=1e-6
        if tcode==1: #  no transformation): x(t)
            y=x
            result=y
        elif tcode==2: # First difference: x(t)-x(t-1)
            y = x.diff()
            result= y
        elif tcode==3:  #Second difference: (x(t)-x(t-1))-(x(t-1)-x(t-2))
            y = x.diff()
            result= y ** 2
        elif tcode==4:    #Natural log: ln(x)
            if min(x) < small:
                y=np.nan
            else :
                y=np.log(x)
            result= y
        elif tcode==5:   #First difference of natural log: ln(x)-ln(x-1)
            y = np.log(x) - np.log(x.shift())
            result=y
        elif tcode==6:   #First difference of natural log: ln(x)-ln(x-1)
            #y = np.log(x) - np.log(x.shift())
            ln_diff1 = np.log(x) - np.log(x.shift())
            ln_diff2 = np.log(x.shift()) - np.log(x.shift(periods=2))
            result=ln_diff1 - ln_diff2
            #result=y**2 
        elif tcode==7 :  #First difference of percent change: (x(t)/x(t-1)-1)-(x(t-1)/x(t-2)-1)
            y=(x/x.shift(1) - 1) - (x.shift(1)/x.shift(2) - 1)
            result=y
        return result
    
    def macro_current(self, current):
        current_tcode = list(current.values[0])[:]
        current_ticker = current.columns.tolist()[:]
        current1 = current.iloc[1:] # without transformation code
    
        yt=[]                #Initialize output variable                   
        N=current1.shape[1]  #Number of series kept
        for i in range(0,N):
            dum=self.transxf(current1.iloc[:,i],current_tcode[i])
            yt.append(dum)

        trans_data=pd.DataFrame(yt).T
        trans_data.columns=current1.columns
        trans_data.index = current1.index
        return trans_data
    
    def macro_predict8(self, month):
        month["dp"] = np.log(month["D12"]) - np.log(month["Index"])
        month["ep"] = np.log(month["E12"]) - np.log(month["Index"])
        month['tms'] = month['lty'] - month['tbl']
        month['dfy'] = month['BAA'] - month['AAA']
        month_new = month[self.variable]
        yt=[]                #Initialize output variable                   
        N=month_new.shape[1]  #Number of series kept
        for i in range(0,N):   
            dum=self.transxf(month_new.iloc[:,i],self.tcodes[i])
            yt.append(dum)
        trans_macro = pd.DataFrame(yt).T
        return trans_macro 
    
    def merge_data(self, trans_data,trans_macro):
        current_trans = trans_data
        macro8_trans = trans_macro

        current_trans = current_trans.loc['1/1/1980':'12/1/2022']
        macro8_trans = macro8_trans.loc['198001':'202212']
        macro8_trans.set_index(current_trans.index, inplace =True)
        total_macro = pd.concat([current_trans, macro8_trans], axis=1)
        return total_macro 
    
    def missing_macro(self, total_macro):
        total_macro.drop(columns = ['ACOGNO'], inplace = True)
        missing_cols = ['CP3Mx','COMPAPFFx']
        for i in missing_cols:
            total_macro[i].fillna(total_macro[i].mean, inplace = True)
            
        return total_macro

In [94]:
class FirmCollect():
    def __init__(self,):
        self.char_list = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 
                        'chcsho','chempia', 'chfeps', 'chinv', 'chmom', 'chnanalyst', 'chpmia', 'chtx', 
                        'cinvest', 'convind', 'currat', 'depr', 'disp', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'fgr5yr', 'gma', 'grcapx', 'grltnoa', 
                        'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'ipo', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mve', 
                        'mve_ia', 'nanalyst', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 
                        'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 
                        'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sfe', 'sgr', 'sgrvol', 'sin', 'sp', 'std_dolvol', 'std_turn', 'stdacc', 'stdcf', 'sue', 
                        'tang', 'tb', 'turn', 'zerotrade']
        self.char_core =['acc', 'agr', 'beta', 'bm', 'cash','cashpr', 'cfp','chatoia', 'chcsho','chfeps',
                        'chinv', 'chmom','chpmia', 'chtx','currat', 'depr','dy', 'ear', 'ep', 'gma',
                        'grcapx', 'grltnoa','ill', 'indmom', 'invest','lev', 'lgr', 'maxret', 'mom12m', 'mom1m',
                        'mom36m','mve','nincr','orgcap','pchgm_pchsale','pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga','retvol', 'roaq', 
                        'roavol', 'roeq','salecash', 'saleinv','sgr','sp', 'std_dolvol', 'std_turn', 'turn',]
        self.info_list = ['jdate','fyear','year','jyear','permno','ticker','comnam','exchcd','exchname',
                          'siccd','indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]
        self.reduce_char = ['absacc','acc','aeavol','age','agr','beta','betasq','bm','bm_ia',
                            'cash','cashdebt','cashpr','cfp','cfp_ia','chatoia','chcsho','chempia',
                            'chinv','chmom','chpmia','chtx','cinvest','convind','currat','depr',
                            'divi','divo','dolvol','dy','ear','egr','ep','gma','grcapx','grltnoa',
                            'herf','hire','idiovol','indmom','invest','ipo','lev','lgr','mom12m',
                            'mom1m','mom36m','mom6m','ms','mve','mve_ia','nincr','operprof','orgcap',
                            'pchcapx_ia','pchcurrat','pchdepr','pchgm_pchsale','pchquick','pchsale_pchinvt',
                            'pchsale_pchrect','pchsale_pchxsga','pchsaleinv','pctacc','pricedelay','ps',
                            'quick','rd','rd_mve','rd_sale','roaq','roavol','roeq','roic','rsup','salecash',
                            'saleinv','salerec','securedind','sgr','sgrvol','sin','sp','stdacc','stdcf','sue',
                            'tang','tb','turn']
    
    def drop_missing(self,df):
        df = df[self.info_list + self.char_core]
        df = df.dropna(subset=self.char_core)
        return df 
    
    def replace_missing_median(self,df):
        for col in self.char_list:
            df[col] = df.groupby('jdate')[col].transform(lambda x: x.fillna(x.median()))
        df2.dropna(axis=1, inplace=True)
        return df
        
        
    def cs_Rank(self,df):
        if df.shape[1] >100:
            print("Orginal 103 Variables")
            df[self.reduce_char] = df.groupby('jdate')[self.reduce_char].transform(lambda x: (x.rank(method='average')/x.count())-0.5)
        else:
            print("49 Variable")
            df[self.char_core] = df.groupby('jdate')[self.char_core].transform(lambda x: (x.rank(method='average')/x.count())-0.5)
        df.sort_values(['permno','jdate'], inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df 
    
    def get_label(self,df):
        df['predicted_return'] = df.groupby('permno')['ret_adj_ex'].shift(-1)
        df = df.dropna()
        df.index = df['jdate']
        del(df['jdate'])
        df = df.sort_index(ascending=True)
        return df 

In [50]:
# Collect 49 Firm-level Characteristics 
firm_collect = FirmCollect()
df1 = firm.copy()
df1 = firm_collect.drop_missing(df1)
df1 = firm_collect.cs_Rank(df1)
df1 = firm_collect.get_label(df1)

df1.to_pickle('firm_df49.pkl')

In [102]:
# Collecr 103 Firm-level Characteristics range [-0.5, 0.5]
firm_collect = FirmCollect()
df2 = firm.copy()
df2 = firm_collect.replace_missing_median(df2)
df2 = firm_collect.cs_Rank(df2)
df2 = firm_collect.get_label(df2)

df2.to_pickle('firm_df103.pkl')

Orginal 103 Variables


In [100]:
macro = MacroCollect()
trans_data = macro.macro_current(current)
trans_macro = macro.macro_predict8(month)
total_macro = macro.merge_data(trans_data,trans_macro)
total_macro = macro.missing_macro(total_macro)

total_macro.to_pickle("macro_df.pkl")

/Users/scarlett/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Validate MacroData

In [9]:
total_macro.isnull().sum().sort_values(ascending = False)

RPI           0
BAA           0
WPSFD49207    0
EXCAUSx       0
EXUSUKx       0
             ..
USTPU         0
SRVPRD        0
NDMANEMP      0
DMANEMP       0
svar          0
Length: 134, dtype: int64

In [10]:
df[firm_collect.char_core].describe()

,acc,agr,beta,bm,cash,cashpr,cfp,chatoia,chcsho,chfeps,...,roaq,roavol,roeq,salecash,saleinv,sgr,sp,std_dolvol,std_turn,turn
count,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,...,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000,210205.000000
mean,0.001287,0.001436,0.001149,0.000843,0.001204,0.001655,0.001257,0.001214,0.001088,0.001493,...,0.001798,0.000792,0.001686,0.001234,0.001206,0.001446,0.000800,0.000549,0.000859,0.001053
std,0.288581,0.288579,0.288674,0.288507,0.288588,0.288652,0.288460,0.288595,0.288685,0.274607,...,0.288522,0.288613,0.288502,0.288627,0.288559,0.288588,0.288521,0.288603,0.288502,0.288464
min,-0.498503,-0.498503,-0.498423,-0.498267,-0.498503,-0.498471,-0.498501,-0.498473,-0.498480,-0.498483,...,-0.498503,-0.498503,-0.498501,-0.498503,-0.498503,-0.498503,-0.498503,-0.498501,-0.498503,-0.498503
25%,-0.248673,-0.248447,-0.248865,-0.249049,-0.248756,-0.248315,-0.248476,-0.248700,-0.250794,-0.231818,...,-0.247899,-0.249175,-0.248044,-0.248727,-0.248673,-0.248387,-0.249077,-0.249319,-0.248996,-0.248760
50%,0.001377,0.001529,0.001199,0.000770,0.001259,0.001767,0.001326,0.001259,0.000945,0.021858,...,0.001901,0.000768,0.001742,0.001326,0.001259,0.001553,0.000780,0.000000,0.000821,0.001044
75%,0.251152,0.251227,0.251131,0.250611,0.250958,0.251645,0.250890,0.251055,0.251073,0.159218,...,0.251634,0.250716,0.251534,0.251152,0.250951,0.251232,0.250594,0.250408,0.250656,0.250765
max,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000


In [11]:
trans_data_val = trans_data.copy()
trans_data_val = trans_data_val.loc['12/1/1966':'11/1/2016']
trans_data_val_col = trans_data_val.columns.tolist()

markus_col = markus.columns.tolist()

def common(a,b): 
    c = [value for value in a if value in b] 
    return c
common=common(trans_data_val_col,markus_col)

markus_new = markus[common]
trans_data_new = trans_data_val[common]

markus_new.set_index(trans_data_new.index, inplace = True)

for i in markus_new.columns.tolist():
    print(i,trans_data_new[i].corr(markus_new[i]))

RPI 0.9965293312516555
W875RX1 0.9954494450598225
DPCERA3M086SBEA 0.99739532880218
CMRMTSPLx 0.9992681466001035
RETAILx 0.9995679557116284
INDPRO 0.9960689255001538
IPFPNSS 0.9938968508517416
IPFINAL 0.994219999524016
IPCONGD 0.9954796483522897
IPDCONGD 0.9979826897977029
IPNCONGD 0.9913227641928706
IPBUSEQ 0.9944142411238619
IPMAT 0.9973310742752494
IPDMAT 0.9970634667856069
IPNMAT 0.9968492388111351
IPMANSICS 0.9955382099245155
IPB51222S 0.9970791975562837
IPFUELS 0.9993516101578338
CUMFNS 0.9952213676892806
HWI 0.8385797445383888
HWIURATIO 0.9276791842764218
CLF16OV 0.9980404978595487
CE16OV 0.9995425397448461
UNRATE 0.9954467132414585
UEMPMEAN 0.9938633093685549
UEMPLT5 0.9988685120969618
UEMP5TO14 0.9994922917707812
UEMP15OV 0.9992001675975166
UEMP15T26 0.9977864179990886
UEMP27OV 0.9995380829879272
CLAIMSx 0.999667903458004
PAYEMS 0.9990535010682224
USGOOD 0.9997069354260556
CES1021000001 0.9994517578704234
USCONS 0.9996052123558187
MANEMP 0.9998789964703054
DMANEMP 0.99993439513

In [12]:
trans_macro8 = trans_macro.copy()
trans_macro8 =trans_macro8.loc['196612': '201611']

variable = ['dp','ep','b/m', 'ntis','tbl','tms','dfy','svar',]
Macro_markus = markus[variable]

Macro_markus.reset_index(inplace = True) 
trans_macro8.reset_index(inplace = True) 

for i in Macro_markus.columns.tolist()[1:]:
    print(Macro_markus[i].corr(trans_macro8[i]))

0.99954746936974
0.9997966571584396
0.9221973057312423
0.9997256637755887
0.9999984092084161
0.9999999261109774
1.0
0.9999999993189497
